In [ ]:
# deep_optimal_stopping.ipynb
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import cholesky
from tqdm.notebook import tqdm
from torch.utils.data import TensorDataset, DataLoader
from scipy.stats import norm
import os
import random
from torch import nn
# setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
#fnn
class StoppingPolicy(torch.nn.Module):
    """
    3-layer neural network with batch normalization and Xavier initialization
    """
    def __init__(self, input_dim, hidden_dim=40):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1),
            torch.nn.Sigmoid()
        )
        # Xavier
        for layer in self.net:
            if isinstance(layer, torch.nn.Linear):
                torch.nn.init.xavier_normal_(layer.weight)

    def forward(self, x):
        return self.net(x)

# can also try cnn algorithm
# if want to use cnn, remember to change:
# self.policies = [
#    CNNStoppingPolicy(input_dim, hidden_dim).to(device)
#    for _ in range(steps + 1)
#]
class CNNStoppingPolicy(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim=40):
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv1d(1, 16, kernel_size=3, padding=1),  # [B, 1, d+1] → [B, 16, d+1]
            torch.nn.BatchNorm1d(16),
            torch.nn.ReLU(),
            torch.nn.Conv1d(16, 32, kernel_size=3, padding=1),  # [B, 16, d+1] → [B, 32, d+1]
            torch.nn.BatchNorm1d(32),
            torch.nn.ReLU(),
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Flatten(),  # flattened to [B, 32 * (d+1)]
            torch.nn.Linear(32 * input_dim, hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim, 1),
            torch.nn.Sigmoid()
        )
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, torch.nn.Linear) or isinstance(m, torch.nn.Conv1d):
                torch.nn.init.xavier_uniform_(m.weight)

    def forward(self, x):
        # x shape: [batch_size, input_dim] → reshape to [B, 1, d+1]
        x = x.unsqueeze(1)
        x = self.conv(x)
        x = self.fc(x)
        return x


In [ ]:
def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
class PathGenerator:
    """Abstract base class for path generation"""
    def __init__(self, d, steps, T):
        self.d = d          # dimension
        self.steps = steps  # time steps
        self.T = T         # total time

    def generate(self, n_paths):
        raise NotImplementedError

In [ ]:
class BlackScholesGenerator(PathGenerator):
    """
    Multidimensional Black-Scholes path generator
    """
    def __init__(self, d, steps, T, r, sigma, rho, div):
        super().__init__(d, steps, T)
        self.r = r
        self.sigma = sigma
        self.rho = rho
        self.div = div
        self.dt = T / steps
        self.drift = (self.r - self.div - 0.5 * self.sigma**2) * self.dt 
        self._build_cov_matrix()
        
    def _build_cov_matrix(self):
        """Construct the covariance matrix"""
        cov = np.eye(self.d) * self.sigma**2
        cov += np.ones((self.d, self.d)) * (self.rho * self.sigma**2)
        np.fill_diagonal(cov, self.sigma**2)

        #  Cholesky 
        self.L = torch.tensor(cholesky(cov, lower=True), device=device, dtype=torch.float32)
        
    def generate(self, n_paths):
        """generate paths """
        paths = torch.zeros(n_paths, self.steps+1, self.d, device=device, dtype=torch.float32)
        paths[:, 0] = 100.0  # initial price
        
        noise = torch.randn(n_paths, self.steps, self.d, device=device)
        
        sqrt_dt = torch.tensor(np.sqrt(self.dt), device=device, dtype=torch.float32)
        with tqdm(total=self.steps, desc="generate Black-Scholes path", leave=False) as pbar:
            for t in range(1, self.steps+1):
                increments = self.drift + (self.L @ noise[:, t-1].T).T* sqrt_dt
                paths[:, t] = paths[:, t-1] * torch.exp(increments)
                pbar.update(1)
                
        return paths

In [ ]:
class DeepOptimalStoppingTrainer:
    """
    Implement the recursive training algorithm from Section 2
    """
    def __init__(self, generator, hidden_dim=40):
        self.generator = generator
        self.policies = self._init_policies(hidden_dim)
        self.optimizers = [torch.optim.Adam(p.parameters(), lr=0.001) 
                          for p in self.policies]
        
    def _init_policies(self, hidden_dim):
        input_dim = self.generator.d + 1  # state+profit
        return [StoppingPolicy(input_dim, hidden_dim).to(device)
                for _ in range(self.generator.steps + 1)]
    
    def train(self, n_paths=8192, epochs=3000, batch_size=2048):
        """Run training using backward recursion"""
        paths = self.generator.generate(n_paths)
        for step in tqdm(reversed(range(self.generator.steps)), desc="Training steps"):
            current_payoff = self._compute_payoffs(paths, start_step=step)[:, step]
            X, y = self._prepare_training_data(paths, current_payoff, step)
            self._train_single_step(step, X, y, epochs, batch_size)
    
    def _compute_payoffs(self, paths):
        """Compute the immediate reward at each time step (to be implemented by subclass)"""
        raise NotImplementedError
    
    def _prepare_training_data(self, paths, current_payoff, step):
        """Prepare training data (including nested Monte Carlo simulation)"""
        current_state = paths[:, step]
        X = torch.cat([current_state, current_payoff.unsqueeze(1)], dim=1)
        
        # Estimate continuation value via nested Monte Carlo (pass starting time step)
        continuation = self._nested_mc(paths, step)
        y = (current_payoff >= continuation).float()
        return X, y

    def _nested_mc(self, paths, step, J=1000):
        n_paths = paths.size(0)
        continuation = torch.zeros(n_paths, device=device)
        
        with tqdm(total=n_paths, desc=f"nested MC(t={step})", leave=False) as main_pbar:
            for i in range(n_paths):
                # Generate J continuation paths from current step
                new_paths = self._generate_continuations(paths[i, step], step, J)
                
                # Apply continuation strategy (starting time step is passed)
                exercise_times = self._apply_policies(new_paths, start_step=step)
                
                # Compute payoffs of continuation paths starting from step
                payoffs = self._compute_continuation_payoffs(new_paths, exercise_times, step)
                
                continuation[i] = payoffs.mean()
                main_pbar.update(1)
        
        return continuation
    
    def _generate_continuations(self, current_state, step, J):
        """Generate continuation paths from current state (to be implemented by subclass)"""
        raise NotImplementedError
    
    def _apply_policies(self, paths, start_step):
        """Apply trained policy network (Eq. (5)) """
        exercise_times = torch.full((paths.size(0),), self.generator.steps, device=device)
        for t in range(start_step + 1, paths.size(1)):
            states = paths[:, t]
            # Dynamically compute payoff at current time step (start_step as reference)
            payoffs = self._compute_payoffs(
                paths[:, :t+1], 
                start_step=start_step
            )[:, t - start_step]  
            
            inputs = torch.cat([states, payoffs.unsqueeze(1)], dim=1)
            stop_probs = self.policies[t](inputs).squeeze()
            stop_decisions = (stop_probs >= 0.5).float()
            
            mask = (exercise_times == self.generator.steps) & (stop_decisions == 1)
            exercise_times[mask] = t
        
        return exercise_times

    def _train_single_step(self, step, X, y, epochs, batch_size):
        """Train policy network at a single time step"""
        dataset = torch.utils.data.TensorDataset(X, y)
        loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
        
        for epoch in tqdm(range(epochs), desc=f"Training step {step}", leave=False):
            for batch_X, batch_y in loader:
                self.optimizers[step].zero_grad()
                pred = self.policies[step](batch_X).squeeze()
                loss = torch.nn.BCELoss()(pred, batch_y)
                loss.backward()
                self.optimizers[step].step()
    
    def compute_lower_bound(self, n_paths=4096000, conf_level=0.95):
        """Compute lower bound and confidence interval (Section 3.1) """
        # Generate independent set of paths
        test_paths = self.generator.generate(n_paths)
        
        # Determine stopping time using trained policy
        exercise_times = self._apply_policies(test_paths, 0)
        
        # Compute payoffs
        payoffs = torch.zeros(n_paths, device=device)
        for i in range(n_paths):
            t = int(exercise_times[i].item())
            payoffs[i] = self._compute_payoffs(test_paths[i:i+1])[0, t]
        
        mean = payoffs.mean().item()
        std = payoffs.std(unbiased=True).item()
        z = 1.96 if conf_level == 0.95 else norm.ppf((1 + conf_level)/2)
        ci_half = z * std / np.sqrt(n_paths)
        
        return {
            'estimate': mean,
            'lower': mean - ci_half,
            'upper': mean + ci_half,
            'std': std
        }
    
    def compute_upper_bound(self, n_paths=1024, J=16384, conf_level=0.95):
        """Compute upper bound and confidence interval"""
        # Generate independent primary paths
        primary_paths = self.generator.generate(n_paths)
        payoffs = self._compute_payoffs(primary_paths, start_step=0)  # shape: [n_paths, steps+1]
        steps = self.generator.steps  
        
        # ============== New: Compute exercise times for primary paths ==============
        # Initialize exercise times, default to maturity (steps)
        exercise_times = torch.full((n_paths,), steps, device=device)
        
        # Traverse each timestep, use policy networks to determine exercise time
        for t in range(steps):
            # Get state and payoff at time t for primary paths
            states = primary_paths[:, t]  # shape: [n_paths, d]
            current_payoff = payoffs[:, t]  # shape: [n_paths]
            
            # Concatenate inputs (state + current payoff)
            inputs = torch.cat([states, current_payoff.unsqueeze(1)], dim=1)
            
            # Use policy network at time t to decide stop
            stop_probs = self.policies[t](inputs).squeeze()
            stop_decisions = (stop_probs >= 0.5).float()  # 0 or 1
            
            # Update exercise times: only affect paths not yet stopped
            mask = (exercise_times == steps) & (stop_decisions == 1)
            exercise_times[mask] = t
        
        # Compute average exercise time
        avg_exercise_time = exercise_times.float().mean().item()
        print(f"Average exercise time: {avg_exercise_time:.2f} (steps), corresponds to: {avg_exercise_time * self.generator.dt:.2f} years")
        # =============================================================================
        
        # Following original code (compute martingale and upper bound)
        M = torch.zeros(n_paths, steps + 1, device=device)
        E_H_prev = payoffs[:, 0].clone()  # Initial expected value
        
        for t in tqdm(range(1, steps+1), desc="Building Martingale"):
            steps_remaining = steps - (t - 1)  
            time_points = steps_remaining + 1  
            cont_paths = torch.zeros(n_paths, J, time_points, self.generator.d, device=device)
            cont_values = torch.zeros(n_paths, J, device=device)
            
            for i in range(n_paths):
                current_state = primary_paths[i, t-1]
                cont_paths[i] = self._generate_continuations(current_state, t-1, J)
                ex_times = self._apply_policies(cont_paths[i], start_step=t-1)
                cont_values[i] = self._compute_continuation_payoffs(
                    cont_paths[i], ex_times, current_step=t-1
                )
            
            E_H_t = cont_values.mean(dim=1)
            delta_M = torch.clamp(payoffs[:, t-1] - E_H_prev, min=0)
            M[:, t] = M[:, t-1] + delta_M
            E_H_prev = E_H_t.clone()
            
            # Debug info (optional)
            avg_cont = E_H_prev.mean().item()
            avg_payoff = payoffs[:, t-1].mean().item()
            print(f"Step {t}: Avg immediate payoff = {avg_payoff:.4f}, Avg continuation value = {avg_cont:.4f}, Mean ΔM = {delta_M.mean().item():.4f}")
        
        # Compute adjusted payoffs
        adjusted_payoffs = torch.max(payoffs - M[:, :payoffs.size(1)], dim=1)[0]
        
        # Statistical results
        mean_est = adjusted_payoffs.mean().item()
        std_est = adjusted_payoffs.std(unbiased=True).item()
        z = 1.96 if conf_level == 0.95 else norm.ppf((1+conf_level)/2)
        ci_half = z * std_est / np.sqrt(n_paths)
        
        return {
            'estimate': mean_est,
            'lower': mean_est - ci_half,
            'upper': mean_est + ci_half,
            'std': std_est,
            'avg_exercise_time': avg_exercise_time  # New: return average exercise time
        }

In [ ]:
class BermudanMaxCallTrainer(DeepOptimalStoppingTrainer):
    """Implement Bermudan Max-Call experiment (Section 4.1)"""
    def _compute_payoffs(self, paths, start_step=0):
        # paths shape: (n_paths, L, d), where L is the actual number of time steps
        max_values, _ = torch.max(paths, dim=2)  # (n_paths, L)
        L = paths.size(1) 
        # Construct time vector: start_time, start_time + dt, ..., start_time + (L - 1) * dt
        time_points = start_step * self.generator.dt + torch.arange(L, device=device, dtype=torch.float32) * self.generator.dt
        discounts = torch.exp(-self.generator.r * time_points)
        return (max_values - 100).clamp(min=0) * discounts  # K=100

    def _generate_continuations(self, current_state, step, J):
        """Generate continuation paths from step to steps (inclusive)"""
        steps_remaining = self.generator.steps - step 
        # Need to generate steps_remaining + 1 time points (from step to steps, inclusive)
        new_paths = torch.zeros(J, steps_remaining + 1, self.generator.d, device=device)
        new_paths[:, 0] = current_state
        sqrt_dt = torch.sqrt(torch.tensor(self.generator.dt, 
                                    device=device, dtype=torch.float32))
        for t in range(1, steps_remaining + 1): 
            Z = torch.randn(J, self.generator.d, device=device)
            increments = self.generator.drift + (self.generator.L @ Z.T).T * sqrt_dt
            new_paths[:, t] = new_paths[:, t-1] * torch.exp(increments)
        return new_paths
    
    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        # new_paths shape: (batch_size, L, d)， L = steps_remaining + 1
        batch_size, L, _ = new_paths.shape
        time_points = current_step * self.generator.dt + torch.arange(L, device=device, dtype=torch.float32) * self.generator.dt
        discounts = torch.exp(-self.generator.r * time_points)[1:]
        
        max_values, _ = torch.max(new_paths, dim=2)  # (batch_size, L)
        payoffs = (max_values[:, 1:] - 100).clamp(min=0) * discounts 
        
        selected_payoffs = torch.zeros(batch_size, device=device)
        for i in range(batch_size):
            absolute_t = int(exercise_times[i].item())
            if absolute_t == self.generator.steps: 
                selected_payoffs[i] = payoffs[i, -1]
            else:
                relative_t = absolute_t - (current_step + 1)
                if relative_t < 0 or relative_t >= (L - 1):
                    raise ValueError(f"Invalid exercise time {absolute_t} at step {current_step}")
                selected_payoffs[i] = payoffs[i, relative_t]
        
        return selected_payoffs

In [ ]:
class MBRCGenerator(BlackScholesGenerator):
    """Path generator with dividends and barrier monitoring"""
    def __init__(self, d, steps, T, r, sigma, rho, div_dates, div_rates, barriers):
        super().__init__(d, steps, T, r, sigma, rho, div=0)
        self.div_dates = div_dates  # List of dividend payment time indices
        self.div_rates = div_rates  # Dividend rates for each asset
        self.barriers = barriers    # Barrier level (percentage)

    def generate(self, n_paths):
        paths = super().generate(n_paths)
        for t in self.div_dates:
            if t >= self.steps:
                raise ValueError(f"Dividend date {t} exceeds path steps {self.steps}")
            paths[:, t+1:] *= (1 - torch.tensor(self.div_rates, device=device))
        return paths

    def track_barriers(self, paths):
        """Monitor barrier events within the local time window"""
        barrier_hit = torch.zeros_like(paths[:, :, 0], dtype=torch.bool)
        for t_local in range(paths.size(1)): 
            current_min = torch.min(paths[:, t_local, :], dim=1).values
            barrier_hit[:, t_local] = (current_min <= self.barriers * 100)
        return barrier_hit

In [ ]:
class CallableMBRCTrainer(DeepOptimalStoppingTrainer):
    """MBRC trainer minimizing issuer's cost"""
    def __init__(self, generator, coupon_rate, nominal=100):
        super().__init__(generator)
        self.coupon = coupon_rate * generator.T / generator.steps
        self.nominal = nominal

    def _compute_payoffs(self, paths, start_step=0):
        barrier_indicator = self.generator.track_barriers(paths)
        n_paths, total_time_steps, _ = paths.shape
        
        start_time = start_step * self.generator.dt
        time_points = torch.linspace(
            start_time,
            self.generator.T,
            total_time_steps,
            device=device
        )
        discounts = torch.exp(-self.generator.r * time_points)
        
        payoffs = torch.zeros(n_paths, total_time_steps, device=device)
        
        for t in range(total_time_steps):
            global_t = start_step + t
            global_t = min(global_t, self.generator.steps)  
            
            
            coupon_part = self.coupon * (global_t + 1)
            
            if global_t == self.generator.steps:
                final_prices = paths[:, t, :]
                min_price = torch.min(final_prices, dim=1).values
                # use torch.minimum to replace torch.min
                payoff = torch.where(
                    barrier_indicator[:, global_t],
                    torch.minimum(min_price, torch.tensor(self.nominal, device=device)),
                    self.nominal
                )
            else:
                payoff = self.nominal
            
            payoffs[:, t] = (coupon_part + payoff) * discounts[t]
        
        return payoffs
    
    def _generate_continuations(self, current_state, step, J):
        """Generate continuation paths with dividends"""
        new_paths = torch.zeros(J, self.generator.steps-step, self.generator.d, device=device)
        new_paths[:, 0] = current_state
        
        for t in range(1, self.generator.steps-step):
            Z = torch.randn(J, self.generator.d, device=device)
            increments = self.generator.drift + (self.generator.L @ Z.T).T * np.sqrt(self.generator.dt)
            new_paths[:, t] = new_paths[:, t-1] * torch.exp(increments)
            
            # Apply dividend payments
            if (step + t) in self.generator.div_dates:
                new_paths[:, t:] *= (1 - torch.tensor(self.generator.div_rates, device=device))
        
        return new_paths

    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        barrier_indicator = self.generator.track_barriers(new_paths)
        batch_size, steps_remaining, _ = new_paths.shape
        
        start_time = current_step * self.generator.dt
        time_points = torch.linspace(
            start_time,
            self.generator.T,
            steps_remaining + 1,
            device=device
        )
        discounts = torch.exp(-self.generator.r * time_points)[1:]
        
        payoffs = torch.zeros(batch_size, device=device)
        for i in range(batch_size):
            t = int(exercise_times[i].item())
            relative_t = t - current_step - 1
            
            if relative_t < 0 or relative_t >= steps_remaining:
                raise ValueError(f"Invalid exercise time {t} at step {current_step}")
            
            global_t = min(current_step + relative_t + 1, self.generator.steps)
            
            coupon_part = self.coupon * (global_t + 1)
            
            if global_t == self.generator.steps:
                final_prices = new_paths[i, relative_t, :]
                min_price = torch.min(final_prices)
                payoff = torch.where(
                    barrier_indicator[i, relative_t],
                    torch.minimum(min_price, torch.tensor(self.nominal, device=device)),
                    self.nominal
                )
            else:
                payoff = self.nominal
            
            discounted_payoff = (coupon_part + payoff) * discounts[relative_t]
            payoffs[i] = discounted_payoff
        
        return payoffs

In [ ]:
class FBMGenerator(PathGenerator):
    """Fractional Brownian motion (fBM) generator with state embedding"""
    def __init__(self, H, steps, T):
        super().__init__(steps+1, steps, T) 
        self.H = H
        self._build_cov_matrix()
    
    def _build_cov_matrix(self):
        t = np.linspace(0, self.T, self.steps+1)
        cov = 0.5 * (
            t[:, None]**(2 * self.H) 
            + t[None, :]**(2 * self.H) 
            - np.abs(t[:, None] - t[None, :])**(2 * self.H)
        )
        np.fill_diagonal(cov, cov.diagonal() + 1e-6)
        
        self.cov_matrix = torch.tensor(
            cov.astype(np.float32), 
            device=device,
            dtype=torch.float32      
        )
        
        # Cholesky decomposition
        self.L = torch.linalg.cholesky(self.cov_matrix)
        
    def generate(self, n_paths):
        paths = torch.zeros(n_paths, self.steps+1, device=device, dtype=torch.float32)
        for i in range(n_paths):
            Z = torch.randn(self.steps+1, device=device, dtype=torch.float32)
            paths[i] = self.L @ Z
        
        # Key: generate 3D embedded paths with shape (n_paths, steps + 1, steps + 1)
        embedded = torch.zeros(n_paths, self.steps+1, self.steps+1, device=device)
        for t in range(self.steps+1):
            embedded[:, t, :t+1] = paths[:, :t+1] 
        return embedded

class FBMTrainer(DeepOptimalStoppingTrainer):
    def __init__(self, H, steps, T, hidden_dim=140):
        generator = FBMGenerator(H, steps, T)
        super().__init__(generator)
        input_dim = (steps + 1) + 1  # state(steps+1) + profit(1)
        self.policies = [
            StoppingPolicy(input_dim, hidden_dim).to(device)
            for _ in range(steps + 1)
        ]
        self.optimizers = [torch.optim.Adam(p.parameters(), lr=0.001) for p in self.policies]

    def _prepare_training_data(self, paths, current_payoff, step):
        current_state = paths[:, step, :] 
        X = torch.cat([current_state, current_payoff.unsqueeze(1)], dim=1)  #  (n_paths, steps+2)
        continuation = self._nested_mc(paths, step)
        y = (current_payoff >= continuation).float()
        return X, y

    def _apply_policies(self, paths, start_step):
        exercise_times = torch.full((paths.size(0),), self.generator.steps, device=device)
        for t in range(start_step + 1, self.generator.steps + 1):
            if t >= paths.size(1):
                break  
            
            states = paths[:, t, :]
            payoffs = self._compute_payoffs(paths[:, :t+1, :], start_step=start_step)[:, t - start_step]
            inputs = torch.cat([states, payoffs.unsqueeze(1)], dim=1)
            stop_probs = self.policies[t](inputs).squeeze()
            stop_decisions = (stop_probs >= 0.5).float()
            
            mask = (exercise_times == self.generator.steps) & (stop_decisions == 1)
            exercise_times[mask] = t
        return exercise_times

    def _compute_payoffs(self, paths, start_step=0):
        """Extract FBM values from 3D paths"""
        n_paths, steps_remaining_plus_1, features = paths.shape
        payoffs = torch.zeros(n_paths, steps_remaining_plus_1, device=device)
        for t in range(steps_remaining_plus_1):
            payoffs[:, t] = paths[:, t, t]  
        return payoffs

    def _generate_continuations(self, current_state, step, J):
        """Generate conditional paths with strictly matched number of steps"""
        t = step
        total_steps = self.generator.steps
     
        future_steps = total_steps - t  
        
        cov = self.generator.cov_matrix  #  (total_steps+1, total_steps+1)
        
        # Split the covariance matrix
        Sigma11 = cov[:t+1, :t+1]          # Covariance of historical paths (t+1, t+1)
        Sigma12 = cov[:t+1, t+1:t+1+future_steps]  # History-future covariance (t+1, future_steps)
        Sigma22 = cov[t+1:t+1+future_steps, t+1:t+1+future_steps]  # Future path covariance (future_steps, future_steps)
        
        Sigma22_1 = Sigma22 - Sigma12.T @ torch.linalg.inv(Sigma11) @ Sigma12
        L22 = torch.linalg.cholesky(Sigma22_1)
        
        # random noise
        Z = torch.randn(J, future_steps, device=device)
        
        current_state_truncated = current_state[:t+1]
        mean_part = current_state_truncated @ torch.linalg.pinv(Sigma11) @ Sigma12  #  (1, future_steps)
        random_part = (L22 @ Z.T).T  #  (J, future_steps)
        
        continuation_values = mean_part + random_part  # (J, future_steps)
        
        cond_paths = torch.zeros(J, total_steps+1, total_steps+1, device=device)
        cond_paths[:, :t+1, 0] = current_state_truncated.expand(J, -1)
        cond_paths[:, t+1:t+1+future_steps, t+1:t+1+future_steps] = continuation_values.unsqueeze(-1)
        
        return cond_paths
    
    def _compute_continuation_payoffs(self, new_paths, exercise_times, current_step):
        """Compute continuation payoffs by directly extracting FBM values """
        payoffs = torch.zeros(new_paths.size(0), device=device)
        for i in range(new_paths.size(0)):
            t = int(exercise_times[i].item())
            relative_t = t - current_step - 1
            if relative_t < 0 or relative_t >= new_paths.size(1):
                raise ValueError(f"Invalid exercise time {t} at step {current_step}")
            payoffs[i] = new_paths[i, relative_t, relative_t]  
        return payoffs

    def compute_upper_bound(self, n_paths=1024, J=16384, conf_level=0.95):
        #  Generate main paths
        primary_paths = self.generator.generate(n_paths)
        payoffs = self._compute_payoffs(primary_paths)
        
        # Initialize the martingale process
        M = torch.zeros(n_paths, self.generator.steps+1, device=device)
        
        # Compute martingale increments
        for t in tqdm(range(self.generator.steps), desc="Computing upper bound (Martingale)"):
            steps_remaining = self.generator.steps - t
            cont_paths = torch.zeros(
                n_paths, J, steps_remaining, self.generator.d, 
                device=device
            )
            
            # generate continuation paths
            for i in range(n_paths):
                full_cont_path = self._generate_continuations(primary_paths[i, t], t, J)
                cont_paths[i] = full_cont_path[:, t+1:t+1+steps_remaining, :]
            
            # Compute the continuation value
            cont_values = torch.zeros(n_paths, device=device)
            for i in range(n_paths):
                ex_times = self._apply_policies(cont_paths[i], t)
                cont_values[i] = self._compute_continuation_payoffs(cont_paths[i], ex_times, t).mean()
            
            # update Martingale
            current_payoff = payoffs[:, t]
            delta_M = (current_payoff >= cont_values).float() * (current_payoff - cont_values)
            M[:, t+1] = M[:, t] + delta_M
        
        # compute adjusted payoffs
        adjusted = payoffs - M
        max_values, _ = torch.max(adjusted, dim=1)
        
        # Compute statistics and confidence intervals
        mean = max_values.mean().item()
        std = max_values.std(unbiased=True).item()
        z = 1.96 if conf_level == 0.95 else norm.ppf((1 + conf_level)/2)
        ci_half = z * std / np.sqrt(n_paths)
        
        return {
            'estimate': mean,
            'lower': mean - ci_half,
            'upper': mean + ci_half,
            'std': std
        }

In [ ]:
# Experiment Execution
def run_Bermudan_experiment():
    params = {
        "d": 3,
        "steps": 9,
        "T": 3.0,
        "r": 0.05,
        "sigma": 0.2,
        "rho": 0.0,
        "div": 0.1
    }
    
    # Initialize the generator and trainer
    bs_gen = BlackScholesGenerator(**params)
    print("BlackScholesGenerator steps:", bs_gen.steps)
    trainer = BermudanMaxCallTrainer(bs_gen, hidden_dim=40+params["d"])
    
    # Training (1500 steps, 4000 paths per batch)
    print("Start training...")
    trainer.train(n_paths=40, epochs=15+params["d"],batch_size=2048)#8192,3000
    
    # Compute lower and upper bound
    print("Computing lower bound...")
    lb_result = trainer.compute_lower_bound(n_paths=2000)#4096000
    print("Computing upper bound...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=1300) #1024，16384
    
    # print results
    print(f"\nBermudan results:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['lower']:.3f}, {ub_result['upper']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")


def run_mbrc_experiment():
    params = {
        "d": 2,
        "steps": 10,       # 2 weeks
        "T": 1.0,
        "r": 0.00,
        "sigma": 0.2,
        "rho": 0.0,
        "div_dates": [5],  
        "div_rates": [0.05]*2,
        "barriers": 0.7      
    }
    
    mbrc_gen = MBRCGenerator(**params)
    trainer = CallableMBRCTrainer(mbrc_gen, coupon_rate=0.07, nominal=100,hidden_dim=40+params["d"])
    
    print("Training Callable MBRC policies...")
    trainer.train(n_paths=1000, epochs=800+params["d"])#8192,3000
    
    # Compute lower and upper bound
    print("Computing lower bound...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)#4096000
    print("Computing upper bound...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=1000)#1024，16384
    
    
    # print results
    print(f"\nMBRC results:")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['upper']:.3f}, {ub_result['lower']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")


def run_fbm_experiment(H=0.7):
    assert 0 < H < 1, "Hurst must be between (0, 1) "
    params = {
        "H": H,
        "steps": 20,#100
        "T": 1.0
    }
    
    fbm_gen = FBMGenerator(**params)
    trainer = FBMTrainer(**params)
    
    print(f"Training FBM (H={H}) policies...")
    trainer.train(n_paths=4000, epochs=1500)#6000
    
    print("Computing lower bound...")
    lb_result = trainer.compute_lower_bound(n_paths=2000000)
    print("Computing upper bound...")
    ub_result = trainer.compute_upper_bound(n_paths=1024, J=13000 if H != 0.5 else 26000)#16384，32768
    
    print(f"\nFBM optimal stopping results(H={H}):")
    print(f"Lower bound: {lb_result['estimate']:.3f} ({lb_result['lower']:.3f}, {lb_result['upper']:.3f})")
    print(f"Upper bound: {ub_result['estimate']:.3f} ({ub_result['lower']:.3f}, {ub_result['upper']:.3f})")
    print(f"95% Confidence Interval: [{lb_result['upper']:.3f}, {ub_result['lower']:.3f}]")
    print(f"Point Estimate: {(lb_result['estimate'] + ub_result['estimate'])/2:.3f}")

In [ ]:
if __name__ == "__main__":
    
    print("========= Bermudan Experiment =========")
    run_Bermudan_experiment()
    
    '''
    print("========= Callable MBRC Experiment =========")
    run_mbrc_experiment()
    
    
    print("\n========= FBM Experiment =========")
    for H in [0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
             0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]:
        run_fbm_experiment(H)''
    '''